# OSM downloader gui example

## Load libraries


In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, time
import numpy as np
import pandas as pd
    
from pathlib import Path #we'll import Path object with `.ls` method added later
from pprint import pprint as pp

import pdb

import matplotlib.pyplot as plt
%matplotlib inline

# ignore warnings
import warnings
if not sys.warnoptions:
    warnings.simplefilter('ignore')
    
# Don't generate bytecode
sys.dont_write_bytecode = True

In [ ]:
import holoviews as hv
import xarray as xr

from holoviews import opts
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
from holoviews.streams import Stream, param
from holoviews import streams
import geoviews as gv
import geoviews.feature as gf
from geoviews import tile_sources as gvts


# import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cf

hv.notebook_extension('bokeh')
hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%d'

# Dashboards
import param as pm, panel as pn
pn.extension()

In [ ]:
# Geoviews visualization default options
H,W, = 250,250
opts.defaults(
    opts.RGB(height=H, width=W, tools=['hover'], active_tools=['wheel_zoom']),
    opts.Image(height=H, width=W, tools=['hover'], active_tools=['wheel_zoom'], framewise=True),#axiswise=True ),
    opts.Points( tools=['hover'], active_tools=['wheel_zoom']),
)

## Set up additional library path

In [ ]:
# Add the utils directory to the search path
SP_ROOT = Path.home()/'Playground/ContextNet'
SP_LIBS = SP_ROOT/'scripts' # to be changed to 'src'
# LIBS_DIR = Path('../src').absolute()
DIRS_TO_ADD = [SP_LIBS]#, LIBS_DIR]
for p in DIRS_TO_ADD:
    assert p.exists()
    
    if str(p) not in sys.path:
        sys.path.insert(0, str(p))
        print(f"Added to sys.path: {p}")

# pp(sys.path)
    

In [ ]:
from output_helpers import print_mro as mro, nprint, Path
import SpacenetPath as spp
import spacenet_globals as spg
# from output_helpers import Path #.ls method is added to Path class

## Step 1: Explore your dataset
    

In [ ]:
city = 'vegas'
rgb8_dir = spp.sample_rgb8_dirs[city]
mask_dir = spp.sample_mask_dirs[city]
sp_vec_dir = spp.sample_road_vec_dirs[city]
osm_mask_dir = spp.sample_mask_dirs[city]

In [ ]:
rgb_fns = sorted([rgb8_dir/fn for fn in rgb8_dir.ls() if Path(fn).suffix in ['.tif', '.tiff']])
mask_fns = sorted([mask_dir/fn for fn in mask_dir.ls() if Path(fn).suffix in ['.tif', '.tiff']])

In [ ]:
for rgb_fn, mask_fn in zip(rgb_fns, mask_fns):
    assert rgb_fn.exists() and mask_fn.exists()
    

## OSM Downloader

In [ ]:
import xarray as xr
import osmnx as ox
import rasterio as rio
import geopandas as gpd
from rasterio.plot import reshape_as_image

In [ ]:
basemap = hv.element.tiles.tile_sources['EsriImagery']()

In [ ]:
class OSMDL(pm.Parameterized):
    fn = pm.FileSelector(path=str(rgb8_dir/"*.tif"))
   
    osm_log = pm.String(default="", 
                        label="OSM log",
                        doc="simple log string about OSM download status")
    osm_dl_count = pm.Number(0)#, precedence=0) #inivisible widget
    
    action_dl = pm.Action(lambda x: x.param.trigger('action_dl'), label='click to download osm')
    action_click = pm.Action(lambda x: x.param.trigger('action_click'))#, precedence=-1)
    
    def __init__(self, **params):
        super().__init__(**params)
        self.osm_g = None
        self.osm_edges = gpd.GeoDataFrame()
        print('initialized')
        
    @param.depends('action_click', watch=True)
    def echo(self):
        print('here')
        self.osm_log = 'clicked'
        print('action clicked')
    
    @param.depends('action_dl', watch=True)
    def _download_osm(self):
        print('Started downloading osm data')
        with rio.open(self.fn) as ds:
            bounds = ds.bounds
        north, south, east, west = bounds.top, bounds.bottom, bounds.right, bounds.left
        self.osm_g = ox.graph_from_bbox(north, south, east, west)
        self.osm_edges = ox.graph_to_gdfs(self.osm_g, edges=True, nodes=False)
                                                      
        print("OSM data downloaded")
        self.osm_dl_count += 1
        self.osm_log = f'OSM data downloaded: {bounds}, {len(self.osm_edges)}'
        
    def show_osm_edges(self):
        return basemap * gv.Path(self.osm_edges)
    
                           
                           
                           
     

In [ ]:
ex = OSMDL()
col = pn.Column()
for p in ex.param:
    col.append(pn.panel(ex.param[p]))
col.servable()
# pn.panel(ex.param.).servable()

In [ ]:
ex._d